## 기본 환경 설정

In [3]:
!python --version

Python 3.10.12


In [ ]:
!wget https://www.python.org/ftp/python/3.6.9/Python-3.6.9.tgz
!tar xvfz Python-3.6.9.tgz
!Python-3.6.9/configure
!make
!sudo make install

In [5]:
!python --version

Python 3.6.9


In [ ]:
!pip install mxnet

In [ ]:
!pip install gluonnlp pandas tqdm

In [ ]:
!pip install sentencepiece

In [ ]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh

In [ ]:
!pip install transformers

In [ ]:
!pip install torch

In [ ]:
!pip install pandas

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [18]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

- torch, nn, F, optim : PyTorch 라이브러리에서 제공되는 모듈, 함수
- Dataset : 데이터 저장, DataLoader : 데이터 관리 및 공급
- gluonnlp : GlunoNLP, 자연어 처리 라이브러리
- numpy : NumPy 라이브러리, 다차원 배열과 행렬 연산에 이용
- tqdm : 진행 상황을 시각적으로 보여주는 라이브러리
- pandas : 데이터 조작 및 분석 라이브러리

In [21]:
# KoBERT
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

# transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

- kobert_tokenizer : 한국어 텍스트를 위한 토크나이저
- BertModel : 구글에서 개발된 BERT 모델, 자연어 처리 모델 라이브러리
- AdamW : [Adam 알고리즘 + 가중치 감쇠] 모델의 복잡도를 줄임
- get_cosine_schedule_with_warmup : 학습률을 조절하는 스케줄러

In [22]:
#GPU 사용
device = torch.device("cuda:0")

- GPU : CPU 보다 빠르게 연산되는 병렬 프로세서
- cuda:0 : 첫 번째 GPU를 의미

In [23]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


- tokenizer : 문장을 토큰으로 분리하여, 인덱스로 변환
- bertmodel : 텍스트의 의미 이해, 특성 추출
- vocal : KoBERT의 Vocabulary 설정, 인덱스 매핑

## 데이터셋 전처리

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
dataset = pd.read_excel('/content/drive/MyDrive/dataset/dataset.xlsx')

In [34]:
dataset.sample(n=1)

,문장,감정
48949,남자 친구랑 관계 후 피임이 잘 되지 않은 것 같아. 임신이면 어떡하지? 응. 병원...,공포


In [35]:
len(dataset)

96865

In [36]:
dataset["감정"].value_counts()

슬픔    25545
분노    16082
공포    15901
놀람    15702
행복    13376
혐오     5429
중립     4830
Name: 감정, dtype: int64

In [37]:
# 정수 변환
# 슬픔(0), 분노(1), 공포(2), 놀람(3), 행복(4), 혐오(5), 중립(6)
dataset.loc[(dataset["감정"] == "슬픔"), "감정"] = 0
dataset.loc[(dataset["감정"] == "분노"), "감정"] = 1
dataset.loc[(dataset["감정"] == "공포"), "감정"] = 2
dataset.loc[(dataset["감정"] == "놀람"), "감정"] = 3
dataset.loc[(dataset["감정"] == "행복"), "감정"] = 4
dataset.loc[(dataset["감정"] == "혐오"), "감정"] = 5
dataset.loc[(dataset["감정"] == "중립"), "감정"] = 6

In [38]:
dataset["감정"].value_counts()

0    25545
1    16082
2    15901
3    15702
4    13376
5     5429
6     4830
Name: 감정, dtype: int64

In [43]:
dataset.sample(n=10)

,문장,감정
45380,직원이 아무 연락도 없이 회사에 출근하지 않아 전화해 보니 여태 자다가 전화를 받았...,1
93121,회사 일로 과로로 쓰러졌는데 회사에서는 나보고 치료비를 내라고 해. 상처를 받았어....,0
89612,부모님과 자녀를 모두 돌보는 나의 상황이 너무 안타깝고 슬퍼. 내가 누군가를 돌볼만...,0
27466,기사볼때마다 그냥 흐뭇하다 ㅋㅋㅋ,4
27961,야 음악 좋다!!!!!다!!!!!다!!!!,4
26865,아이언맨 슈트나 빨랑 개발혀라.,6
79011,자꾸 요즘에는 안 좋은 생각이 들어. 그냥 모든 일에 의욕도 없고 사회생활도 하기 ...,1
79132,올바른정책이 사회적으로 지지받지 못하면 너무답답해. 의견도 내고 나라가 안정되고 좋...,0
1666,어떻게 해야될지 모르겠어요..0,2
76308,성적이 자꾸 하락해서 불안해. 노력은 하는데 성적은 자꾸 떨어지고 있어. 잠을 제대...,2


In [45]:
# train & test 데이터로 분류하기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(dataset, test_size= 0.2, random_state = 0)

- test_size= 0.2 : 80%는 train, 20%는 test로 추출
- shuffle = True : 디폴트값으로 생략 가능
- random_state : 난수의 초기값 설정

In [46]:
len(dataset_train)

77492

In [47]:
len(dataset_test)

19373